In [1]:
from dataclasses import dataclass
import numpy as np
import numba as nb
from numba import njit, prange
from tqdm.auto import tqdm
from numba.typed import List, Dict
from scipy.special import factorial
from prettytable import PrettyTable 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from src.Util import dt_factorial, _dt_factorial, dt_sum, _sort_and_fill, \
    sort_and_fill, binomial_coefficient, _fill_zeros, fill_zeros, _remove_tail, \
    remove_tail
from src.DerivativeTypes import generate_derivative_types, _generate_derivative_types, \
    _generate_derivative_subtypes, generate_derivative_subtypes
from src.Combinatorics import _compute_etas, compute_etas, _compute_zetas, compute_zetas, \
    _compute_sorted_zetas, compute_sorted_zetas, _number_of_representations, number_of_representations
from src.Hashing import _der_type_to_hash, der_type_to_hash, _der_types_to_hashes, der_types_to_hashes
from src.DerivativeBounds import _make_dbound_dict, make_dbound_dict

In [4]:
from src.DerivativeBounds import _compute_cumulated_g_bounds_for_zeta, _compute_cumulated_g_bounds_for_eta, \
    _compute_cumulated_g_bounds_for_j, _compute_cumulated_g_bounds, compute_cumulated_g_bounds_for_zeta, compute_cumulated_g_bounds_for_eta, \
    compute_cumulated_g_bounds_for_j, compute_cumulated_g_bounds, compute_bound_for_alpha, _compute_bound_for_alpha

## Generate Derivative Types

In [5]:
generate_derivative_types(4, 6)

ListType[array(int16, 1d, C)]([[1 1 1 1 0 0], [2 1 1 0 0 0], [2 2 0 0 0 0], [3 1 0 0 0 0], [4 0 0 0 0 0], ...])

In [6]:
[(list(l), sum(l)) for l in generate_derivative_types(4, 6)]

[([1, 1, 1, 1, 0, 0], 4),
 ([2, 1, 1, 0, 0, 0], 4),
 ([2, 2, 0, 0, 0, 0], 4),
 ([3, 1, 0, 0, 0, 0], 4),
 ([4, 0, 0, 0, 0, 0], 4)]

In [7]:
%%timeit
_generate_derivative_types(10, 500)

8.18 ms ± 517 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
generate_derivative_types(10, 500)

167 µs ± 8.64 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [9]:
[(list(l), sum(l)) for l in generate_derivative_subtypes(4, 6)]

[([1, 0, 0, 0, 0, 0], 1),
 ([1, 1, 0, 0, 0, 0], 2),
 ([2, 0, 0, 0, 0, 0], 2),
 ([1, 1, 1, 0, 0, 0], 3),
 ([2, 1, 0, 0, 0, 0], 3),
 ([3, 0, 0, 0, 0, 0], 3),
 ([1, 1, 1, 1, 0, 0], 4),
 ([2, 1, 1, 0, 0, 0], 4),
 ([2, 2, 0, 0, 0, 0], 4),
 ([3, 1, 0, 0, 0, 0], 4),
 ([4, 0, 0, 0, 0, 0], 4)]

In [10]:
%%timeit
_generate_derivative_subtypes(10, 500)

1.2 ms ± 21.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [11]:
%%timeit
generate_derivative_subtypes(10, 500)

510 µs ± 24.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [12]:
%%time
ns = [1, 2, 3, 5, 10, 20] # , 50
ks = [1, 2, 3, 5, 10, 25, 50, 100, 250, 500, 1000]
table = PrettyTable(["k\\n"] + [str(n) for n in ns])
for k in ks:
    table.add_row([str(k)] + [f"{len(generate_derivative_types(n, k)):,}" for n in ns])
print(f'Sorted multi-indices with k entries summing up to n:')
print(table)

Sorted multi-indices with k entries summing up to n:
+------+---+---+---+---+----+-----+
| k\n  | 1 | 2 | 3 | 5 | 10 |  20 |
+------+---+---+---+---+----+-----+
|  1   | 1 | 1 | 1 | 1 | 1  |  1  |
|  2   | 1 | 2 | 2 | 3 | 6  |  11 |
|  3   | 1 | 2 | 3 | 5 | 14 |  44 |
|  5   | 1 | 2 | 3 | 7 | 30 | 192 |
|  10  | 1 | 2 | 3 | 7 | 42 | 530 |
|  25  | 1 | 2 | 3 | 7 | 42 | 627 |
|  50  | 1 | 2 | 3 | 7 | 42 | 627 |
| 100  | 1 | 2 | 3 | 7 | 42 | 627 |
| 250  | 1 | 2 | 3 | 7 | 42 | 627 |
| 500  | 1 | 2 | 3 | 7 | 42 | 627 |
| 1000 | 1 | 2 | 3 | 7 | 42 | 627 |
+------+---+---+---+---+----+-----+
CPU times: user 39.2 ms, sys: 3.78 ms, total: 43 ms
Wall time: 42.8 ms


## Derivative Type Hashing

In [13]:
n = 4
k = 6
[(arr, der_type_to_hash(arr, n, k)) for arr in generate_derivative_types(n, k)]

[(array([1, 1, 1, 1, 0, 0], dtype=int16), 85),
 (array([2, 1, 1, 0, 0, 0], dtype=int16), 22),
 (array([2, 2, 0, 0, 0, 0], dtype=int16), 10),
 (array([3, 1, 0, 0, 0, 0], dtype=int16), 7),
 (array([4, 0, 0, 0, 0, 0], dtype=int16), 4)]

In [14]:
n=25; k=50
hashes = der_types_to_hashes(generate_derivative_types(n, k), n, k)
print(len(hashes), len(set(hashes)))

1958 1958


In [15]:
%%timeit
[_der_type_to_hash(arr, n, k) for arr in generate_derivative_types(n, k)]

91.8 ms ± 2.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
_der_types_to_hashes(generate_derivative_types(n, k), n, k)

22.7 ms ± 341 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
%%timeit
der_types_to_hashes(generate_derivative_types(n, k), n, k)

17.9 ms ± 2.11 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Accessing Derivative Bounds

In [18]:
n = 4; k = 6
make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(5))

DictType[int64,float64]<iv=None>({85: 0.0, 22: 1.0, 10: 2.0, 7: 3.0, 4: 4.0})

In [19]:
%%timeit
n=25; k=50
_make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(1958))

18.6 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit
n=25; k=50
make_dbound_dict(der_types_to_hashes(generate_derivative_types(n, k), n, k), np.arange(1958))

15.7 ms ± 569 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Computing the Derivative Bound

## Compute the Cummulated Bounds of $g$

### Compute $\eta$ , $\zeta$
#### Helper Functions

In [21]:
arr = np.zeros(100, dtype=np.int16)
arr[:5] = 2 * np.arange(5, dtype=np.int16)[::-1]
dt_factorial(arr), np.prod(factorial(arr))

(1393459200, 1393459200.0)

In [22]:
%%timeit
_dt_factorial(arr)

167 µs ± 11.6 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [23]:
%%timeit
np.prod(factorial(arr))

9.72 µs ± 291 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [24]:
%%timeit
dt_factorial(arr)

326 ns ± 19.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [25]:
%%timeit
np.sum(arr)

4.44 µs ± 96.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [26]:
%%timeit
dt_sum(arr)

246 ns ± 12.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [27]:
sort_and_fill(np.arange(3*5, dtype=np.int16).reshape((3, 5)), 10)

array([[ 4,  3,  2,  1,  0,  0,  0,  0,  0,  0],
       [ 9,  8,  7,  6,  5,  0,  0,  0,  0,  0],
       [14, 13, 12, 11, 10,  0,  0,  0,  0,  0]], dtype=int16)

#### Compute $\eta$

In [28]:
[l for l in compute_etas(5, 3, 3, np.array((3, 2, 0), dtype=np.int16))]

[array([[0, 1],
        [3, 1]], dtype=int16),
 array([[0, 2],
        [3, 0]], dtype=int16),
 array([[1, 0],
        [2, 2]], dtype=int16),
 array([[1, 1],
        [2, 1]], dtype=int16),
 array([[1, 2],
        [2, 0]], dtype=int16),
 array([[2, 0],
        [1, 2]], dtype=int16),
 array([[2, 1],
        [1, 1]], dtype=int16),
 array([[2, 2],
        [1, 0]], dtype=int16),
 array([[3, 0],
        [0, 2]], dtype=int16),
 array([[3, 1],
        [0, 1]], dtype=int16)]

In [ ]:
%%timeit
_compute_etas(7, 3, 5, np.array((4, 3, 0), dtype=np.int16))

In [ ]:
%%timeit
compute_etas(7, 3, 5, np.array((4, 3, 0), dtype=np.int16))

#### Compute $\zeta$
For $\zeta := (\zeta^{(1)}, \ldots, \zeta^{(n)}) \in (\mathbb N^k)^n$, there are the following constraints:
- $\zeta^{(i)} = 0$ for $ i < j$
- $\vert \zeta^{(i)} \vert \leq \vert \zeta^{(i+1)} \vert$ for $ i < n$ <-- Constraint on row
- $\sum_{i=j}^{n} \vert \eta^{(i)} \vert \zeta^{(i)} = \alpha$ <-- Constraint on column

In [ ]:
n = 5; k = 10; m = 3; j=2; alpha=np.array((3, 2, 0), dtype=np.int16)
etas = compute_etas(n, m, j, alpha)
zetas = compute_zetas(n, k, j, etas[0], alpha)
etas[0], zetas[0], zetas[0] * etas[0].sum(axis=1)[:, None]

In [ ]:
n = 5; k = 10; m = 3; j=2; alpha=np.array((3, 2, 0), dtype=np.int16)
etas = compute_etas(n, m, j, alpha)
zetas = compute_zetas(n, k, j, etas[2], alpha)
[etas[2]] + [l for l in zetas]

In [ ]:
zetas = compute_sorted_zetas(n, k, j, etas[2], alpha)
[l for l in zetas]

### Cummulative $g$ bounds

In [ ]:
generate_derivative_subtypes(n, k)[-2]

In [ ]:
n = 5
m = 10
k = 10
der_types = generate_derivative_subtypes(n, k)
h_der_type = der_types[-2]
f_der_type = generate_derivative_types(n, m)[1]
g_dbounds = make_dbound_dict(der_types_to_hashes(der_types, n, k), np.ones(len(der_types)))
compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

In [ ]:
%%timeit
_compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

In [ ]:
%%timeit
compute_cumulated_g_bounds(n, m, k, h_der_type, f_der_type, g_dbounds)

In [ ]:
compute_cumulated_g_bounds_for_j.parallel_diagnostics()

## Compute Representations of $\beta$

In [ ]:
binomial_coefficient(10, 2)

In [ ]:
beta = np.zeros(1000)
beta[:4] = np.array((1, 1, 1, 0), dtype=np.int16)
number_of_representations(beta)

In [ ]:
%%timeit
_number_of_representations(beta)

In [ ]:
%%timeit
number_of_representations(beta)

## Compute Bound for $\alpha$

In [ ]:
n = 5
m = 10
k = 10
der_types = generate_derivative_subtypes(n, k)
h_der_type = der_types[-2]
der_typesf = generate_derivative_subtypes(n, m)
f_dbounds = make_dbound_dict(der_types_to_hashes(der_typesf, n, m), np.ones(len(der_typesf)))
g_dbounds = make_dbound_dict(der_types_to_hashes(der_types, n, k), np.ones(len(der_types)))
compute_bound_for_alpha(n, m, k, h_der_type, f_dbounds, g_dbounds)

In [ ]:
%%timeit
compute_bound_for_alpha(n, m, k, h_der_type, f_dbounds, g_dbounds)

In [ ]:
np.arange(1, 11, dtype=np.int16)